In [1]:
import pandas as pd
pd.set_option('display.max_columns', 50000)
import numpy as np
import csv
import json
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv(filepath_or_buffer="./dataset/data.csv")
df.head()

,user_id,username,age,status,sex,orientation,drinks,drugs,height,job,location,pets,smokes,language,new_languages,body_profile,education_level,dropped_out,bio,interests,other_interests,location_preference
0,fffe3100,Edith Lopez,27,single,f,gay,socially,never,66.0,medicine / health,"oakland, california",likes dogs and likes cats,no,"english (fluently), spanish (poorly), sign lan...",interested,athletic,4.0,no,bottom line i love life! i work hard and i lov...,sports,instruments,same state
1,fffe3200,Travis Young,26,single,m,gay,socially,never,68.0,other,"pleasant hill, california",likes dogs,no,"english (fluently), tagalog (okay), french (po...",interested,fit,3.0,no,"i'm a straightforward, genuine, fun loving (i'...",painting,instruments,anywhere
2,fffe3300,Agnes Smith,20,seeing someone,f,bisexual,socially,sometimes,69.0,other,"oakland, california",has dogs and likes cats,sometimes,"english (fluently), sign language (poorly), fr...",interested,fit,2.0,no,mmmmm yummy tacosss. yoga is where it's at. i ...,instruments,dancing,same city
3,fffe3400,Salvador Klaver,27,single,m,bisexual,socially,sometimes,68.0,computer / hardware / software,"daly city, california",likes dogs and likes cats,no,english,not interested,average,3.0,no,i'm a stealth geek. that special mix of techni...,sketching,acting,same city
4,fffe3500,Elana Sewell,22,single,f,bisexual,often,sometimes,68.0,other,"oakland, california",likes dogs and likes cats,yes,english,not interested,average,2.0,yes,with the whisper of the wind i was weaved into...,craft,designing,same city


In [3]:
df.describe()

,age,height,education_level
count,2001.000000,2001.000000,2001.000000
mean,33.072464,68.289855,2.932534
std,10.483189,3.895246,0.812371
min,18.000000,55.000000,1.000000
25%,26.000000,65.000000,2.000000
50%,30.000000,68.000000,3.000000
75%,39.000000,71.000000,3.000000
max,69.000000,83.000000,5.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2001 entries, 0 to 2000
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              2001 non-null   object 
 1   username             2001 non-null   object 
 2   age                  2001 non-null   int64  
 3   status               2001 non-null   object 
 4   sex                  2001 non-null   object 
 5   orientation          2001 non-null   object 
 6   drinks               2001 non-null   object 
 7   drugs                2001 non-null   object 
 8   height               2001 non-null   float64
 9   job                  2001 non-null   object 
 10  location             2001 non-null   object 
 11  pets                 2001 non-null   object 
 12  smokes               2001 non-null   object 
 13  language             2001 non-null   object 
 14  new_languages        2001 non-null   object 
 15  body_profile         2001 non-null   o

In [5]:
df["likes_dogs"] = False
df["likes_cats"] = False
df["has_cats"] = False
df["has_dogs"] = False
df["location_state"] = None
df["location_city"] = None
df["language_json"] = None
df.head()

,user_id,username,age,status,sex,orientation,drinks,drugs,height,job,location,pets,smokes,language,new_languages,body_profile,education_level,dropped_out,bio,interests,other_interests,location_preference,likes_dogs,likes_cats,has_cats,has_dogs,location_state,location_city,language_json
0,fffe3100,Edith Lopez,27,single,f,gay,socially,never,66.0,medicine / health,"oakland, california",likes dogs and likes cats,no,"english (fluently), spanish (poorly), sign lan...",interested,athletic,4.0,no,bottom line i love life! i work hard and i lov...,sports,instruments,same state,False,False,False,False,None,None,None
1,fffe3200,Travis Young,26,single,m,gay,socially,never,68.0,other,"pleasant hill, california",likes dogs,no,"english (fluently), tagalog (okay), french (po...",interested,fit,3.0,no,"i'm a straightforward, genuine, fun loving (i'...",painting,instruments,anywhere,False,False,False,False,None,None,None
2,fffe3300,Agnes Smith,20,seeing someone,f,bisexual,socially,sometimes,69.0,other,"oakland, california",has dogs and likes cats,sometimes,"english (fluently), sign language (poorly), fr...",interested,fit,2.0,no,mmmmm yummy tacosss. yoga is where it's at. i ...,instruments,dancing,same city,False,False,False,False,None,None,None
3,fffe3400,Salvador Klaver,27,single,m,bisexual,socially,sometimes,68.0,computer / hardware / software,"daly city, california",likes dogs and likes cats,no,english,not interested,average,3.0,no,i'm a stealth geek. that special mix of techni...,sketching,acting,same city,False,False,False,False,None,None,None
4,fffe3500,Elana Sewell,22,single,f,bisexual,often,sometimes,68.0,other,"oakland, california",likes dogs and likes cats,yes,english,not interested,average,2.0,yes,with the whisper of the wind i was weaved into...,craft,designing,same city,False,False,False,False,None,None,None


In [6]:
print(df["status"].value_counts())
print("-"*50)
print(df["sex"].value_counts())
print("-"*50)
print(df["orientation"].value_counts())
print("-"*50)
print(df["drinks"].value_counts())
print("-"*50)
print(df["drugs"].value_counts())
print("-"*50)
print(df["smokes"].value_counts())
print("-"*50)
print(df["new_languages"].value_counts())
print("-"*50)
print(df["body_profile"].value_counts())
print("-"*50)
print(df["dropped_out"].value_counts())
print("-"*50)
print(df["interests"].value_counts())
print("-"*50)
print(df["other_interests"].value_counts())
print("-"*50)
print(df["location_preference"].value_counts())
print("-"*50)
print(df["location"].value_counts())
print("-"*50)
print(df["pets"].value_counts())
print("-"*50)
print("-"*50)
print("-"*50)
print(df["likes_dogs"].value_counts())
print("-"*50)
print(df["likes_cats"].value_counts())
print("-"*50)
print(df["has_cats"].value_counts())
print("-"*50)
print(df["has_dogs"].value_counts())
print("-"*50)
print(df["location_state"].value_counts())
print("-"*50)
print(df["location_city"].value_counts())

single            1867
available           66
seeing someone      54
married             14
Name: status, dtype: int64
--------------------------------------------------
m    1139
f     862
Name: sex, dtype: int64
--------------------------------------------------
straight    1736
gay          153
bisexual     112
Name: orientation, dtype: int64
--------------------------------------------------
socially       1393
rarely          256
often           168
not at all      157
very often       16
desperately      11
Name: drinks, dtype: int64
--------------------------------------------------
never        1586
sometimes     397
often          18
Name: drugs, dtype: int64
--------------------------------------------------
no                1630
sometimes          128
yes                 94
when drinking       91
trying to quit      58
Name: smokes, dtype: int64
--------------------------------------------------
not interested         1050
interested              633
somewhat interested    

In [7]:
total_languages = []
total_language_proficiency = []
for ind in df.index:
    
    pets = df["pets"][ind]
            
    if "likes dogs" in pets:
        df.loc[ind,"likes_dogs"] = True
    elif "dislikes dogs" in pets:
        df.loc[ind,"likes_dogs"] = False
    else:
        df.loc[ind,"likes_dogs"] = False

    if "likes cats" in pets:
        df.loc[ind,"likes_cats"] = True
    elif "dislikes cats" in pets:
        df.loc[ind,"likes_cats"] = False
    else:
        df.loc[ind,"likes_cats"] = False

    if "has dogs" in pets:
        df.loc[ind,"has_dogs"] = True
    else:
        df.loc[ind,"has_dogs"] = False

    if "has cats" in pets:
        df.loc[ind,"has_cats"] = True
    else:
        df.loc[ind,"has_cats"] = False

    
    location = df["location"][ind]
    x = location.split(",")
    x[0] = x[0].strip()
    x[1] = x[1].strip()
    df.loc[ind,"location_state"] = x[1]
    df.loc[ind,"location_city"] = x[0]
    
    job = df["job"][ind]
    job = job.replace(" / ","/")
    df.loc[ind,"job"] = job
    
    language = df["language"][ind]
    language_json = []
    for lang in language.split(","):
        lang = lang.replace(" ","")
        lang = lang.replace(")","")
        lang = lang.strip()
        language_and_proficiency = lang.split("(")
        if len(language_and_proficiency) < 2:
            language_and_proficiency.append("fluently")
        if language_and_proficiency[0] not in total_languages:
            total_languages.append(language_and_proficiency[0])
        if language_and_proficiency[1] not in total_language_proficiency:
            total_language_proficiency.append(language_and_proficiency[1])
        lan_dict = {}
        lan_dict["language"] = language_and_proficiency[0]
        lan_dict["proficiency"] = language_and_proficiency[1]
        language_json.append(lan_dict)
    
    df.loc[ind,"language_json"] = json.dumps(language_json)
        
df.head()

,user_id,username,age,status,sex,orientation,drinks,drugs,height,job,location,pets,smokes,language,new_languages,body_profile,education_level,dropped_out,bio,interests,other_interests,location_preference,likes_dogs,likes_cats,has_cats,has_dogs,location_state,location_city,language_json
0,fffe3100,Edith Lopez,27,single,f,gay,socially,never,66.0,medicine/health,"oakland, california",likes dogs and likes cats,no,"english (fluently), spanish (poorly), sign lan...",interested,athletic,4.0,no,bottom line i love life! i work hard and i lov...,sports,instruments,same state,True,True,False,False,california,oakland,"[{""language"": ""english"", ""proficiency"": ""fluen..."
1,fffe3200,Travis Young,26,single,m,gay,socially,never,68.0,other,"pleasant hill, california",likes dogs,no,"english (fluently), tagalog (okay), french (po...",interested,fit,3.0,no,"i'm a straightforward, genuine, fun loving (i'...",painting,instruments,anywhere,True,False,False,False,california,pleasant hill,"[{""language"": ""english"", ""proficiency"": ""fluen..."
2,fffe3300,Agnes Smith,20,seeing someone,f,bisexual,socially,sometimes,69.0,other,"oakland, california",has dogs and likes cats,sometimes,"english (fluently), sign language (poorly), fr...",interested,fit,2.0,no,mmmmm yummy tacosss. yoga is where it's at. i ...,instruments,dancing,same city,False,True,False,True,california,oakland,"[{""language"": ""english"", ""proficiency"": ""fluen..."
3,fffe3400,Salvador Klaver,27,single,m,bisexual,socially,sometimes,68.0,computer/hardware/software,"daly city, california",likes dogs and likes cats,no,english,not interested,average,3.0,no,i'm a stealth geek. that special mix of techni...,sketching,acting,same city,True,True,False,False,california,daly city,"[{""language"": ""english"", ""proficiency"": ""fluen..."
4,fffe3500,Elana Sewell,22,single,f,bisexual,often,sometimes,68.0,other,"oakland, california",likes dogs and likes cats,yes,english,not interested,average,2.0,yes,with the whisper of the wind i was weaved into...,craft,designing,same city,True,True,False,False,california,oakland,"[{""language"": ""english"", ""proficiency"": ""fluen..."


In [8]:
lang_columns = []
for lang in total_languages:
    for prof in total_language_proficiency:
        lang_columns.append(lang +"_"+ prof)
df_language = pd.DataFrame(columns=lang_columns)

for ind in df.index:
    language_json = df["language_json"][ind]
    language_json = json.loads(language_json)
    for lan in language_json:
        column_name = lan["language"] + "_" + lan["proficiency"]
        df_language.loc[ind,column_name] = 1
    
df_language = df_language.fillna(0)
# df_language.head()
df = pd.concat([df,df_language],axis=1)
df.head()

,user_id,username,age,status,sex,orientation,drinks,drugs,height,job,location,pets,smokes,language,new_languages,body_profile,education_level,dropped_out,bio,interests,other_interests,location_preference,likes_dogs,likes_cats,has_cats,has_dogs,location_state,location_city,language_json,english_fluently,english_poorly,english_okay,spanish_fluently,spanish_poorly,spanish_okay,signlanguage_fluently,signlanguage_poorly,signlanguage_okay,tagalog_fluently,tagalog_poorly,tagalog_okay,french_fluently,french_poorly,french_okay,italian_fluently,italian_poorly,italian_okay,german_fluently,german_poorly,german_okay,chinese_fluently,chinese_poorly,chinese_okay,c++_fluently,c++_poorly,c++_okay,hebrew_fluently,hebrew_poorly,hebrew_okay,swedish_fluently,swedish_poorly,swedish_okay,hawaiian_fluently,hawaiian_poorly,hawaiian_okay,irish_fluently,irish_poorly,irish_okay,russian_fluently,russian_poorly,russian_okay,korean_fluently,korean_poorly,korean_okay,arabic_fluently,arabic_poorly,arabic_okay,portuguese_fluently,portuguese_poorly,portuguese_okay,japanese_fluently,japanese_poorly,japanese_okay,other_fluently,other_poorly,other_okay,thai_fluently,thai_poorly,thai_okay,esperanto_fluently,esperanto_poorly,esperanto_okay,croatian_fluently,croatian_poorly,croatian_okay,farsi_fluently,farsi_poorly,farsi_okay,greek_fluently,greek_poorly,greek_okay,ukrainian_fluently,ukrainian_poorly,ukrainian_okay,hindi_fluently,hindi_poorly,hindi_okay,sanskrit_fluently,sanskrit_poorly,sanskrit_okay,swahili_fluently,swahili_poorly,swahili_okay,icelandic_fluently,icelandic_poorly,icelandic_okay,bengali_fluently,bengali_poorly,bengali_okay,maori_fluently,maori_poorly,maori_okay,slovak_fluently,slovak_poorly,slovak_okay,norwegian_fluently,norwegian_poorly,norwegian_okay,latin_fluently,latin_poorly,latin_okay,vietnamese_fluently,vietnamese_poorly,vietnamese_okay,czech_fluently,czech_poorly,czech_okay,yiddish_fluently,yiddish_poorly,yiddish_okay,urdu_fluently,urdu_poorly,urdu_okay,turkish_fluently,turkish_poorly,turkish_okay,lisp_fluently,lisp_poorly,lisp_okay,indonesian_fluently,indonesian_poorly,indonesian_okay,slovenian_fluently,slovenian_poorly,slovenian_okay,polish_fluently,polish_poorly,polish_okay,danish_fluently,danish_poorly,danish_okay,ancientgreek_fluently,ancientgreek_poorly,ancientgreek_okay,afrikaans_fluently,afrikaans_poorly,afrikaans_okay,dutch_fluently,dutch_poorly,dutch_okay,albanian_fluently,albanian_poorly,albanian_okay,bulgarian_fluently,bulgarian_poorly,bulgarian_okay,finnish_fluently,finnish_poorly,finnish_okay,hungarian_fluently,hungarian_poorly,hungarian_okay,mongolian_fluently,mongolian_poorly,mongolian_okay,catalan_fluently,catalan_poorly,catalan_okay,tamil_fluently,tamil_poorly,tamil_okay,basque_fluently,basque_poorly,basque_okay,cebuano_fluently,cebuano_poorly,cebuano_okay,gujarati_fluently,gujarati_poorly,gujarati_okay,romanian_fluently,romanian_poorly,romanian_okay,tibetan_fluently,tibetan_poorly,tibetan_okay,armenian_fluently,armenian_poorly,armenian_okay,malay_fluently,malay_poorly,malay_okay,ilongo_fluently,ilongo_poorly,ilongo_okay,persian_fluently,persian_poorly,persian_okay
0,fffe3100,Edith Lopez,27,single,f,gay,socially,never,66.0,medicine/health,"oakland, california",likes dogs and likes cats,no,"english (fluently), spanish (poorly), sign lan...",interested,athletic,4.0,no,bottom line i love life! i work hard and i lov...,sports,instruments,same state,True,True,False,False,california,oakland,"[{""language"": ""english"", ""proficiency"": ""fluen...",1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,fffe3200,Travis Young,26,single,m,gay,socially,never,68.0,other,"pleasant hill, california",likes dogs,no,"englis

In [9]:
print(df["location_state"].value_counts())
print("-"*50)
print(df["location_city"].value_counts())

california    1997
new york         1
ohio             1
vietnam          1
arizona          1
Name: location_state, dtype: int64
--------------------------------------------------
san francisco      911
oakland            250
berkeley           146
san mateo           56
palo alto           45
                  ... 
north hollywood      1
moss beach           1
phoenix              1
petaluma             1
foster city          1
Name: location_city, Length: 70, dtype: int64


In [10]:
label_object = {}
categorical_columns = [
    'status',
    'sex',
    'orientation',
    'drinks',
    'drugs',
    'smokes',
    'new_languages',
    'body_profile',
    'dropped_out',
    'interests',
    'other_interests',
    'location_preference',
    'likes_dogs',
    'likes_cats',
    'has_cats',
    'has_dogs',
    'location_state',
    'location_city'
]
for col in categorical_columns:
    labelencoder = LabelEncoder()
    labelencoder.fit(df[col])
    df[col] = labelencoder.fit_transform(df[col])
    label_object[col] = labelencoder
    
df.head()

,user_id,username,age,status,sex,orientation,drinks,drugs,height,job,location,pets,smokes,language,new_languages,body_profile,education_level,dropped_out,bio,interests,other_interests,location_preference,likes_dogs,likes_cats,has_cats,has_dogs,location_state,location_city,language_json,english_fluently,english_poorly,english_okay,spanish_fluently,spanish_poorly,spanish_okay,signlanguage_fluently,signlanguage_poorly,signlanguage_okay,tagalog_fluently,tagalog_poorly,tagalog_okay,french_fluently,french_poorly,french_okay,italian_fluently,italian_poorly,italian_okay,german_fluently,german_poorly,german_okay,chinese_fluently,chinese_poorly,chinese_okay,c++_fluently,c++_poorly,c++_okay,hebrew_fluently,hebrew_poorly,hebrew_okay,swedish_fluently,swedish_poorly,swedish_okay,hawaiian_fluently,hawaiian_poorly,hawaiian_okay,irish_fluently,irish_poorly,irish_okay,russian_fluently,russian_poorly,russian_okay,korean_fluently,korean_poorly,korean_okay,arabic_fluently,arabic_poorly,arabic_okay,portuguese_fluently,portuguese_poorly,portuguese_okay,japanese_fluently,japanese_poorly,japanese_okay,other_fluently,other_poorly,other_okay,thai_fluently,thai_poorly,thai_okay,esperanto_fluently,esperanto_poorly,esperanto_okay,croatian_fluently,croatian_poorly,croatian_okay,farsi_fluently,farsi_poorly,farsi_okay,greek_fluently,greek_poorly,greek_okay,ukrainian_fluently,ukrainian_poorly,ukrainian_okay,hindi_fluently,hindi_poorly,hindi_okay,sanskrit_fluently,sanskrit_poorly,sanskrit_okay,swahili_fluently,swahili_poorly,swahili_okay,icelandic_fluently,icelandic_poorly,icelandic_okay,bengali_fluently,bengali_poorly,bengali_okay,maori_fluently,maori_poorly,maori_okay,slovak_fluently,slovak_poorly,slovak_okay,norwegian_fluently,norwegian_poorly,norwegian_okay,latin_fluently,latin_poorly,latin_okay,vietnamese_fluently,vietnamese_poorly,vietnamese_okay,czech_fluently,czech_poorly,czech_okay,yiddish_fluently,yiddish_poorly,yiddish_okay,urdu_fluently,urdu_poorly,urdu_okay,turkish_fluently,turkish_poorly,turkish_okay,lisp_fluently,lisp_poorly,lisp_okay,indonesian_fluently,indonesian_poorly,indonesian_okay,slovenian_fluently,slovenian_poorly,slovenian_okay,polish_fluently,polish_poorly,polish_okay,danish_fluently,danish_poorly,danish_okay,ancientgreek_fluently,ancientgreek_poorly,ancientgreek_okay,afrikaans_fluently,afrikaans_poorly,afrikaans_okay,dutch_fluently,dutch_poorly,dutch_okay,albanian_fluently,albanian_poorly,albanian_okay,bulgarian_fluently,bulgarian_poorly,bulgarian_okay,finnish_fluently,finnish_poorly,finnish_okay,hungarian_fluently,hungarian_poorly,hungarian_okay,mongolian_fluently,mongolian_poorly,mongolian_okay,catalan_fluently,catalan_poorly,catalan_okay,tamil_fluently,tamil_poorly,tamil_okay,basque_fluently,basque_poorly,basque_okay,cebuano_fluently,cebuano_poorly,cebuano_okay,gujarati_fluently,gujarati_poorly,gujarati_okay,romanian_fluently,romanian_poorly,romanian_okay,tibetan_fluently,tibetan_poorly,tibetan_okay,armenian_fluently,armenian_poorly,armenian_okay,malay_fluently,malay_poorly,malay_okay,ilongo_fluently,ilongo_poorly,ilongo_okay,persian_fluently,persian_poorly,persian_okay
0,fffe3100,Edith Lopez,27,3,0,1,4,0,66.0,medicine/health,"oakland, california",likes dogs and likes cats,0,"english (fluently), spanish (poorly), sign lan...",0,1,4.0,0,bottom line i love life! i work hard and i lov...,25,13,2,1,1,0,0,1,40,"[{""language"": ""english"", ""proficiency"": ""fluen...",1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,fffe3200,Travis Young,26,3,1,1,4,0,68.0,other,"pleasant hill, california",likes dogs,0,"english (fluently), tagalog (okay), french (po...",0,4,3.0,0,"i'm a straightforward, genuine, fun loving (i'...

In [11]:
cleaned = df.job.str.split('/', expand=True).stack()
job_df = pd.get_dummies(cleaned,prefix="job").groupby(level=0).sum()
df = pd.concat([df,job_df],axis=1)
df.head()

,user_id,username,age,status,sex,orientation,drinks,drugs,height,job,location,pets,smokes,language,new_languages,body_profile,education_level,dropped_out,bio,interests,other_interests,location_preference,likes_dogs,likes_cats,has_cats,has_dogs,location_state,location_city,language_json,english_fluently,english_poorly,english_okay,spanish_fluently,spanish_poorly,spanish_okay,signlanguage_fluently,signlanguage_poorly,signlanguage_okay,tagalog_fluently,tagalog_poorly,tagalog_okay,french_fluently,french_poorly,french_okay,italian_fluently,italian_poorly,italian_okay,german_fluently,german_poorly,german_okay,chinese_fluently,chinese_poorly,chinese_okay,c++_fluently,c++_poorly,c++_okay,hebrew_fluently,hebrew_poorly,hebrew_okay,swedish_fluently,swedish_poorly,swedish_okay,hawaiian_fluently,hawaiian_poorly,hawaiian_okay,irish_fluently,irish_poorly,irish_okay,russian_fluently,russian_poorly,russian_okay,korean_fluently,korean_poorly,korean_okay,arabic_fluently,arabic_poorly,arabic_okay,portuguese_fluently,portuguese_poorly,portuguese_okay,japanese_fluently,japanese_poorly,japanese_okay,other_fluently,other_poorly,other_okay,thai_fluently,thai_poorly,thai_okay,esperanto_fluently,esperanto_poorly,esperanto_okay,croatian_fluently,croatian_poorly,croatian_okay,farsi_fluently,farsi_poorly,farsi_okay,greek_fluently,greek_poorly,greek_okay,ukrainian_fluently,ukrainian_poorly,ukrainian_okay,hindi_fluently,hindi_poorly,hindi_okay,sanskrit_fluently,sanskrit_poorly,sanskrit_okay,swahili_fluently,swahili_poorly,swahili_okay,icelandic_fluently,icelandic_poorly,icelandic_okay,bengali_fluently,bengali_poorly,bengali_okay,maori_fluently,maori_poorly,maori_okay,slovak_fluently,slovak_poorly,slovak_okay,norwegian_fluently,norwegian_poorly,norwegian_okay,latin_fluently,latin_poorly,latin_okay,vietnamese_fluently,vietnamese_poorly,vietnamese_okay,czech_fluently,czech_poorly,czech_okay,yiddish_fluently,yiddish_poorly,yiddish_okay,urdu_fluently,urdu_poorly,urdu_okay,turkish_fluently,turkish_poorly,turkish_okay,lisp_fluently,lisp_poorly,lisp_okay,indonesian_fluently,indonesian_poorly,indonesian_okay,slovenian_fluently,slovenian_poorly,slovenian_okay,polish_fluently,polish_poorly,polish_okay,danish_fluently,danish_poorly,danish_okay,ancientgreek_fluently,ancientgreek_poorly,ancientgreek_okay,afrikaans_fluently,afrikaans_poorly,afrikaans_okay,dutch_fluently,dutch_poorly,dutch_okay,albanian_fluently,albanian_poorly,albanian_okay,bulgarian_fluently,bulgarian_poorly,bulgarian_okay,finnish_fluently,finnish_poorly,finnish_okay,hungarian_fluently,hungarian_poorly,hungarian_okay,mongolian_fluently,mongolian_poorly,mongolian_okay,catalan_fluently,catalan_poorly,catalan_okay,tamil_fluently,tamil_poorly,tamil_okay,basque_fluently,basque_poorly,basque_okay,cebuano_fluently,cebuano_poorly,cebuano_okay,gujarati_fluently,gujarati_poorly,gujarati_okay,romanian_fluently,romanian_poorly,romanian_okay,tibetan_fluently,tibetan_poorly,tibetan_okay,armenian_fluently,armenian_poorly,armenian_okay,malay_fluently,malay_poorly,malay_okay,ilongo_fluently,ilongo_poorly,ilongo_okay,persian_fluently,persian_poorly,persian_okay,job_academia,job_administrative,job_artistic,job_banking,job_biz dev,job_clerical,job_computer,job_construction,job_craftsmanship,job_education,job_engineering,job_entertainment,job_executive,job_financial,job_government,job_hardware,job_health,job_hospitality,job_law,job_legal services,job_management,job_marketing,job_media,job_medicine,job_military,job_musical,job_other,job_political,job_rather not say,job_real estate,job_retired,job_sales,job_science,job_software,job_student,job_tech,job_transportation,job_travel,job_unemployed,job_writer
0,fffe3100,Edith Lopez,27,3,0,1,4,0,66.0,medicine/health,"oakland, california",likes dogs and likes cats,0,"english (fluently), spanish (poorly), sign lan...",0,1,4.0,0,bottom line i love life! i work hard and i lov...,25,13,2,1,1,0,0,1,40,"[{""language"": ""english"", ""proficiency"": ""fluen...",1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0

In [12]:
redundant_columns = ['username','job','location','pets','language','bio','language_json']
df = df.drop(redundant_columns,axis=1)
df.head()

,user_id,age,status,sex,orientation,drinks,drugs,height,smokes,new_languages,body_profile,education_level,dropped_out,interests,other_interests,location_preference,likes_dogs,likes_cats,has_cats,has_dogs,location_state,location_city,english_fluently,english_poorly,english_okay,spanish_fluently,spanish_poorly,spanish_okay,signlanguage_fluently,signlanguage_poorly,signlanguage_okay,tagalog_fluently,tagalog_poorly,tagalog_okay,french_fluently,french_poorly,french_okay,italian_fluently,italian_poorly,italian_okay,german_fluently,german_poorly,german_okay,chinese_fluently,chinese_poorly,chinese_okay,c++_fluently,c++_poorly,c++_okay,hebrew_fluently,hebrew_poorly,hebrew_okay,swedish_fluently,swedish_poorly,swedish_okay,hawaiian_fluently,hawaiian_poorly,hawaiian_okay,irish_fluently,irish_poorly,irish_okay,russian_fluently,russian_poorly,russian_okay,korean_fluently,korean_poorly,korean_okay,arabic_fluently,arabic_poorly,arabic_okay,portuguese_fluently,portuguese_poorly,portuguese_okay,japanese_fluently,japanese_poorly,japanese_okay,other_fluently,other_poorly,other_okay,thai_fluently,thai_poorly,thai_okay,esperanto_fluently,esperanto_poorly,esperanto_okay,croatian_fluently,croatian_poorly,croatian_okay,farsi_fluently,farsi_poorly,farsi_okay,greek_fluently,greek_poorly,greek_okay,ukrainian_fluently,ukrainian_poorly,ukrainian_okay,hindi_fluently,hindi_poorly,hindi_okay,sanskrit_fluently,sanskrit_poorly,sanskrit_okay,swahili_fluently,swahili_poorly,swahili_okay,icelandic_fluently,icelandic_poorly,icelandic_okay,bengali_fluently,bengali_poorly,bengali_okay,maori_fluently,maori_poorly,maori_okay,slovak_fluently,slovak_poorly,slovak_okay,norwegian_fluently,norwegian_poorly,norwegian_okay,latin_fluently,latin_poorly,latin_okay,vietnamese_fluently,vietnamese_poorly,vietnamese_okay,czech_fluently,czech_poorly,czech_okay,yiddish_fluently,yiddish_poorly,yiddish_okay,urdu_fluently,urdu_poorly,urdu_okay,turkish_fluently,turkish_poorly,turkish_okay,lisp_fluently,lisp_poorly,lisp_okay,indonesian_fluently,indonesian_poorly,indonesian_okay,slovenian_fluently,slovenian_poorly,slovenian_okay,polish_fluently,polish_poorly,polish_okay,danish_fluently,danish_poorly,danish_okay,ancientgreek_fluently,ancientgreek_poorly,ancientgreek_okay,afrikaans_fluently,afrikaans_poorly,afrikaans_okay,dutch_fluently,dutch_poorly,dutch_okay,albanian_fluently,albanian_poorly,albanian_okay,bulgarian_fluently,bulgarian_poorly,bulgarian_okay,finnish_fluently,finnish_poorly,finnish_okay,hungarian_fluently,hungarian_poorly,hungarian_okay,mongolian_fluently,mongolian_poorly,mongolian_okay,catalan_fluently,catalan_poorly,catalan_okay,tamil_fluently,tamil_poorly,tamil_okay,basque_fluently,basque_poorly,basque_okay,cebuano_fluently,cebuano_poorly,cebuano_okay,gujarati_fluently,gujarati_poorly,gujarati_okay,romanian_fluently,romanian_poorly,romanian_okay,tibetan_fluently,tibetan_poorly,tibetan_okay,armenian_fluently,armenian_poorly,armenian_okay,malay_fluently,malay_poorly,malay_okay,ilongo_fluently,ilongo_poorly,ilongo_okay,persian_fluently,persian_poorly,persian_okay,job_academia,job_administrative,job_artistic,job_banking,job_biz dev,job_clerical,job_computer,job_construction,job_craftsmanship,job_education,job_engineering,job_entertainment,job_executive,job_financial,job_government,job_hardware,job_health,job_hospitality,job_law,job_legal services,job_management,job_marketing,job_media,job_medicine,job_military,job_musical,job_other,job_political,job_rather not say,job_real estate,job_retired,job_sales,job_science,job_software,job_student,job_tech,job_transportation,job_travel,job_unemployed,job_writer
0,fffe3100,27,3,0,1,4,0,66.0,0,0,1,4.0,0,25,13,2,1,1,0,0,1,40,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [13]:
one_hot_object = {}
categorical_columns = [
    'status',
    'orientation',
    'drinks',
    'drugs',
    'smokes',
    'new_languages',
    'body_profile',
    'interests',
    'other_interests',
    'location_preference',
    'location_state',
    'location_city'
]
for col in categorical_columns:
    onehot_encoder = OneHotEncoder()
    col_data  = df[col].values.reshape(-1,1)
    onehot_encoder.fit(col_data)
    onehot_encoded = onehot_encoder.fit_transform(col_data).toarray()
    columns_list = []
    for i in range(onehot_encoded.shape[1]):
        columns_list.append(col+"_"+str(int(i)))
    dfOneHot = pd.DataFrame(onehot_encoded, columns = columns_list) 
    df = pd.concat([df, dfOneHot], axis=1)
    one_hot_object[col] = onehot_encoder
    
df = df.drop(categorical_columns,axis=1)
df.head()

,user_id,age,sex,height,education_level,dropped_out,likes_dogs,likes_cats,has_cats,has_dogs,english_fluently,english_poorly,english_okay,spanish_fluently,spanish_poorly,spanish_okay,signlanguage_fluently,signlanguage_poorly,signlanguage_okay,tagalog_fluently,tagalog_poorly,tagalog_okay,french_fluently,french_poorly,french_okay,italian_fluently,italian_poorly,italian_okay,german_fluently,german_poorly,german_okay,chinese_fluently,chinese_poorly,chinese_okay,c++_fluently,c++_poorly,c++_okay,hebrew_fluently,hebrew_poorly,hebrew_okay,swedish_fluently,swedish_poorly,swedish_okay,hawaiian_fluently,hawaiian_poorly,hawaiian_okay,irish_fluently,irish_poorly,irish_okay,russian_fluently,russian_poorly,russian_okay,korean_fluently,korean_poorly,korean_okay,arabic_fluently,arabic_poorly,arabic_okay,portuguese_fluently,portuguese_poorly,portuguese_okay,japanese_fluently,japanese_poorly,japanese_okay,other_fluently,other_poorly,other_okay,thai_fluently,thai_poorly,thai_okay,esperanto_fluently,esperanto_poorly,esperanto_okay,croatian_fluently,croatian_poorly,croatian_okay,farsi_fluently,farsi_poorly,farsi_okay,greek_fluently,greek_poorly,greek_okay,ukrainian_fluently,ukrainian_poorly,ukrainian_okay,hindi_fluently,hindi_poorly,hindi_okay,sanskrit_fluently,sanskrit_poorly,sanskrit_okay,swahili_fluently,swahili_poorly,swahili_okay,icelandic_fluently,icelandic_poorly,icelandic_okay,bengali_fluently,bengali_poorly,bengali_okay,maori_fluently,maori_poorly,maori_okay,slovak_fluently,slovak_poorly,slovak_okay,norwegian_fluently,norwegian_poorly,norwegian_okay,latin_fluently,latin_poorly,latin_okay,vietnamese_fluently,vietnamese_poorly,vietnamese_okay,czech_fluently,czech_poorly,czech_okay,yiddish_fluently,yiddish_poorly,yiddish_okay,urdu_fluently,urdu_poorly,urdu_okay,turkish_fluently,turkish_poorly,turkish_okay,lisp_fluently,lisp_poorly,lisp_okay,indonesian_fluently,indonesian_poorly,indonesian_okay,slovenian_fluently,slovenian_poorly,slovenian_okay,polish_fluently,polish_poorly,polish_okay,danish_fluently,danish_poorly,danish_okay,ancientgreek_fluently,ancientgreek_poorly,ancientgreek_okay,afrikaans_fluently,afrikaans_poorly,afrikaans_okay,dutch_fluently,dutch_poorly,dutch_okay,albanian_fluently,albanian_poorly,albanian_okay,bulgarian_fluently,bulgarian_poorly,bulgarian_okay,finnish_fluently,finnish_poorly,finnish_okay,hungarian_fluently,hungarian_poorly,hungarian_okay,mongolian_fluently,mongolian_poorly,mongolian_okay,catalan_fluently,catalan_poorly,catalan_okay,tamil_fluently,tamil_poorly,tamil_okay,basque_fluently,basque_poorly,basque_okay,cebuano_fluently,cebuano_poorly,cebuano_okay,gujarati_fluently,gujarati_poorly,gujarati_okay,romanian_fluently,romanian_poorly,romanian_okay,tibetan_fluently,tibetan_poorly,tibetan_okay,armenian_fluently,armenian_poorly,armenian_okay,malay_fluently,malay_poorly,malay_okay,ilongo_fluently,ilongo_poorly,ilongo_okay,persian_fluently,persian_poorly,persian_okay,job_academia,job_administrative,job_artistic,job_banking,job_biz dev,job_clerical,job_computer,job_construction,job_craftsmanship,job_education,job_engineering,job_entertainment,job_executive,job_financial,job_government,job_hardware,job_health,job_hospitality,job_law,job_legal services,job_management,job_marketing,job_media,job_medicine,job_military,job_musical,job_other,job_political,job_rather not say,job_real estate,job_retired,job_sales,job_science,job_software,job_student,job_tech,job_transportation,job_travel,job_unemployed,job_writer,status_0,status_1,status_2,status_3,orientation_0,orientation_1,orientation_2,drinks_0,drinks_1,drinks_2,drinks_3,drinks_4,drinks_5,drugs_0,drugs_1,drugs_2,smokes_0,smokes_1,smokes_2,smokes_3,smokes_4,new_languages_0,new_languages_1,new_languages_2,body_profile_0,body_profile_1,body_profile_2,body_profile_3,body_profile_4,body_profile_5,body_profile_6,body_profile_7,body_profile_8,body_profile_9,body_profile_10,body_profile_11,interests_0,interests_1,interests_2,interests_3,interests_4,interests_5,interests_6,interests_

In [14]:
df_backup = df.copy(deep=True)
# user_ids = df.loc[:,"user_id"]
df= df.drop(["user_id"],axis=1)

### Check this

[Clustering Tutorial](https://towardsdatascience.com/dating-algorithms-using-machine-learning-and-ai-814b68ecd75e)

In [15]:
"""
# Setting the amount of clusters to test out
cluster_cnt = [i for i in range(2, 125, 1)]

# Establishing empty lists to store the scores for the evaluation metrics
s_scores = []

db_scores = []

# Looping through different iterations for the number of clusters
for i in cluster_cnt:
    
    # Hierarchical Agglomerative Clustering with different number of clusters
    hac = AgglomerativeClustering(n_clusters=i)
    
    hac.fit(df)
    
    cluster_assignments = hac.labels_
    
    ## KMeans Clustering with different number of clusters
    #k_means = KMeans(n_clusters=i)
    
    #k_means.fit(df_pca)
    
    #cluster_assignments = k_means.predict(df_pca)
    
    # Appending the scores to the empty lists    
    s_scores.append(silhouette_score(df, cluster_assignments))
    
    db_scores.append(davies_bouldin_score(df, cluster_assignments))

"""
# Plotting graph
"""
def plot_evaluation(y, x=cluster_cnt):
    
    # Creating a DataFrame for returning the max and min scores for each cluster
    df = pd.DataFrame(columns=['Cluster Score'], index=[i for i in range(2, len(y)+2)])
    df['Cluster Score'] = y
    
    print('Max Value:\nCluster #', df[df['Cluster Score']==df['Cluster Score'].max()])
    print('\nMin Value:\nCluster #', df[df['Cluster Score']==df['Cluster Score'].min()])
    
    # Plotting out the scores based on cluster count
    plt.figure(figsize=(16,6))
    plt.style.use('ggplot')
    plt.plot(x,y)
    plt.xlabel('# of Clusters')
    plt.ylabel('Score')
    plt.show()
    
# Running the function on the list of scores
plot_evaluation(s_scores)

plot_evaluation(db_scores)

"""

"\ndef plot_evaluation(y, x=cluster_cnt):\n    \n    # Creating a DataFrame for returning the max and min scores for each cluster\n    df = pd.DataFrame(columns=['Cluster Score'], index=[i for i in range(2, len(y)+2)])\n    df['Cluster Score'] = y\n    \n    print('Max Value:\nCluster #', df[df['Cluster Score']==df['Cluster Score'].max()])\n    print('\nMin Value:\nCluster #', df[df['Cluster Score']==df['Cluster Score'].min()])\n    \n    # Plotting out the scores based on cluster count\n    plt.figure(figsize=(16,6))\n    plt.style.use('ggplot')\n    plt.plot(x,y)\n    plt.xlabel('# of Clusters')\n    plt.ylabel('Score')\n    plt.show()\n    \n# Running the function on the list of scores\nplot_evaluation(s_scores)\n\nplot_evaluation(db_scores)\n\n"

In [16]:
# Instantiating HAC
hac = AgglomerativeClustering(n_clusters=125)

# Fitting
hac.fit(df)

# Getting cluster assignments
cluster_assignments = hac.labels_

# Assigning the clusters to each profile
df['cluster_id'] = cluster_assignments

# Viewing the dating profiles with cluster assignments
df.head()

,age,sex,height,education_level,dropped_out,likes_dogs,likes_cats,has_cats,has_dogs,english_fluently,english_poorly,english_okay,spanish_fluently,spanish_poorly,spanish_okay,signlanguage_fluently,signlanguage_poorly,signlanguage_okay,tagalog_fluently,tagalog_poorly,tagalog_okay,french_fluently,french_poorly,french_okay,italian_fluently,italian_poorly,italian_okay,german_fluently,german_poorly,german_okay,chinese_fluently,chinese_poorly,chinese_okay,c++_fluently,c++_poorly,c++_okay,hebrew_fluently,hebrew_poorly,hebrew_okay,swedish_fluently,swedish_poorly,swedish_okay,hawaiian_fluently,hawaiian_poorly,hawaiian_okay,irish_fluently,irish_poorly,irish_okay,russian_fluently,russian_poorly,russian_okay,korean_fluently,korean_poorly,korean_okay,arabic_fluently,arabic_poorly,arabic_okay,portuguese_fluently,portuguese_poorly,portuguese_okay,japanese_fluently,japanese_poorly,japanese_okay,other_fluently,other_poorly,other_okay,thai_fluently,thai_poorly,thai_okay,esperanto_fluently,esperanto_poorly,esperanto_okay,croatian_fluently,croatian_poorly,croatian_okay,farsi_fluently,farsi_poorly,farsi_okay,greek_fluently,greek_poorly,greek_okay,ukrainian_fluently,ukrainian_poorly,ukrainian_okay,hindi_fluently,hindi_poorly,hindi_okay,sanskrit_fluently,sanskrit_poorly,sanskrit_okay,swahili_fluently,swahili_poorly,swahili_okay,icelandic_fluently,icelandic_poorly,icelandic_okay,bengali_fluently,bengali_poorly,bengali_okay,maori_fluently,maori_poorly,maori_okay,slovak_fluently,slovak_poorly,slovak_okay,norwegian_fluently,norwegian_poorly,norwegian_okay,latin_fluently,latin_poorly,latin_okay,vietnamese_fluently,vietnamese_poorly,vietnamese_okay,czech_fluently,czech_poorly,czech_okay,yiddish_fluently,yiddish_poorly,yiddish_okay,urdu_fluently,urdu_poorly,urdu_okay,turkish_fluently,turkish_poorly,turkish_okay,lisp_fluently,lisp_poorly,lisp_okay,indonesian_fluently,indonesian_poorly,indonesian_okay,slovenian_fluently,slovenian_poorly,slovenian_okay,polish_fluently,polish_poorly,polish_okay,danish_fluently,danish_poorly,danish_okay,ancientgreek_fluently,ancientgreek_poorly,ancientgreek_okay,afrikaans_fluently,afrikaans_poorly,afrikaans_okay,dutch_fluently,dutch_poorly,dutch_okay,albanian_fluently,albanian_poorly,albanian_okay,bulgarian_fluently,bulgarian_poorly,bulgarian_okay,finnish_fluently,finnish_poorly,finnish_okay,hungarian_fluently,hungarian_poorly,hungarian_okay,mongolian_fluently,mongolian_poorly,mongolian_okay,catalan_fluently,catalan_poorly,catalan_okay,tamil_fluently,tamil_poorly,tamil_okay,basque_fluently,basque_poorly,basque_okay,cebuano_fluently,cebuano_poorly,cebuano_okay,gujarati_fluently,gujarati_poorly,gujarati_okay,romanian_fluently,romanian_poorly,romanian_okay,tibetan_fluently,tibetan_poorly,tibetan_okay,armenian_fluently,armenian_poorly,armenian_okay,malay_fluently,malay_poorly,malay_okay,ilongo_fluently,ilongo_poorly,ilongo_okay,persian_fluently,persian_poorly,persian_okay,job_academia,job_administrative,job_artistic,job_banking,job_biz dev,job_clerical,job_computer,job_construction,job_craftsmanship,job_education,job_engineering,job_entertainment,job_executive,job_financial,job_government,job_hardware,job_health,job_hospitality,job_law,job_legal services,job_management,job_marketing,job_media,job_medicine,job_military,job_musical,job_other,job_political,job_rather not say,job_real estate,job_retired,job_sales,job_science,job_software,job_student,job_tech,job_transportation,job_travel,job_unemployed,job_writer,status_0,status_1,status_2,status_3,orientation_0,orientation_1,orientation_2,drinks_0,drinks_1,drinks_2,drinks_3,drinks_4,drinks_5,drugs_0,drugs_1,drugs_2,smokes_0,smokes_1,smokes_2,smokes_3,smokes_4,new_languages_0,new_languages_1,new_languages_2,body_profile_0,body_profile_1,body_profile_2,body_profile_3,body_profile_4,body_profile_5,body_profile_6,body_profile_7,body_profile_8,body_profile_9,body_profile_10,body_profile_11,interests_0,interests_1,interests_2,interests_3,interests_4,interests_5,interests_6,interests_7,intere

In [17]:
cluster_df = pd.DataFrame(pd.concat([df_backup["user_id"],df["cluster_id"]], axis=1),columns=["user_id","cluster_id"])

In [18]:
cluster_df.to_csv(path_or_buf="./cluster_df.csv", index=False, quotechar='"',float_format='%.15f', quoting=csv.QUOTE_NONNUMERIC, encoding='utf-8')